In [ ]:
from PIL import Image,ImageOps,ImageStat,ImageDraw
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow  import keras as keras
from keras import Model
from keras import layers
from keras import losses
from keras.models import Sequential
from keras.models import Model # basic class for specifying and training a neural network
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten, Activation, Conv2D
#from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import  ModelCheckpoint
from keras.utils import np_utils # utilities for one-hot encoding of ground truth values
from keras.optimizers import Adam
from keras.utils import to_categorical
import os,os.path
import sys
import math
import sys
import numpy as np
import random
tf.enable_eager_execution()
tf.set_random_seed(173)
#psp_fileName = "D:/_SAR_Kubinka/__FRAGM1856X.JPG._PSP"
#image_fileName = "D:/_SAR_Kubinka/__FRAGM1856X.JPG"
#newMask_fileName = "D:/_SAR_Kubinka/new_mask_class_"
#fragm_dir = "D:/_SAR_Kubinka/fragments"
#acc_file = 'D:/_SAR_Kubinka/accuracy.txt'
#result_fileName = "D:/_SAR_Kubinka/classification_result.BMP"
#marks_filename = 'D:/_SAR_Kubinka/class_marks.txt'
#img_len = 1024

img_file_name = image_fileName
#дополнять массив цветами по мере необходимости\n",
t_color = ['blue', 'red', 'yellow', 'green' , 'violet']
temple_file = Image.open(img_file_name)
colors = {}
#сборка границ прямоугольникв, из которых будут вырезаться области (если их на изображении несколько)
classRects=[]
curColor = ""
class_numb = '021'
rect_flag  = False
   
img = Image.new('RGB', temple_file.size, color=0)
draw  = ImageDraw.Draw(img)
fp = open(psp_fileName)
print("all was opened")
for k, txt in enumerate(fp):
    if k<2:
        continue
    zz, t = txt.split('=')
    if t.find('Pline') >= 0:
        x2y = []
        rect_flag = False
        continue
    if t.find('Rectangle') >= 0:
        x2y = []
        rect_flag = True
        continue
    if t.find('Pen') >= 0:
        p = t.split(',')
        curColor = p[2]
        if p[2] not in  colors:
            colors.update({p[2]: len(colors)})
   
    t = t.split(' ')
    if t[0] == '':
        if rect_flag:
            x2y.insert(1,(x2y[0][0],x2y[1][1]))
            x2y.append( (x2y[2][0], x2y[0][1]))  
        idx = colors.get(curColor) 
        if idx ==  len(classRects):
            classRects.append([x2y])
        else:
            classRects[idx].append(x2y)
        #print(classRects)   
        draw.polygon(x2y, outline = t_color[idx], fill = t_color[idx])
        continue
    x2y.append((int(t[0]), int(t[1])))
print("masks were drawen")
img.save(newMask_fileName + class_numb +".BMP")
img.close()
fp.close()


In [ ]:
win_size = step_len * 2 + 1
bound = math.ceil(math.sqrt(2)*win_size-win_size)
dir_name = fragm_dir
labels=[]
nmb_fragm = 0
print(bound)
im = Image.open(img_file_name)
for i in range(len(colors)):
    for j in range(len(classRects[i])):
        
        x0 = classRects[i][j][0][0]
        y0 = classRects[i][j][0][1]
        x_len =  classRects[i][j][1][1]-classRects[i][j][0][1]
        y_len =  classRects[i][j][2][0]-classRects[i][j][1][0]
        x_amount = int(x_len/step_len)
        y_amount = int(y_len/step_len)
        for k in range(x_amount):
            for l in range(y_amount):
                labels.append(i)
                im2=im.crop((x0 + k * step_len -3, y0+ l * step_len -3, x0 + k * step_len +3 + win_size,y0+ l * step_len +3 + win_size)) 
                imfnam=dir_name+"\\+++fragm{:05d}.jpg".format(nmb_fragm); 
                nmb_fragm+=1;
                im2.save(imfnam)
im.close()

In [ ]:
transAmount = 6;
dlin   = 0

def load_CNN_train_augment(dir_name1):
    ll=[]           #пустой список имен входных файлов JPG 
    for file in os.listdir(dir_name1):
        if file.endswith(".jpg"): ll.append(file)
    dlin   = len(ll)     
    dlin0  = int(len(ll)/2)  #будем пропускать нечетные элементы
    #dlin=int(dlin0*(var+1) )  #0 ->просто чтение, 1 ->+90, 2 ->+180, 3 ->+270 дополняем массив
    train_x = np.zeros((dlin * transAmount, win_size, win_size, 3), dtype='float32')
    print('train_x_augm.shape=',train_x.shape)
    train_z = np.zeros((dlin * transAmount, 1), dtype='float32')
    dlin = dlin * transAmount
    k=0
    for file in ll:
        im  = Image.open(dir_name+"\\"+file)
        train_x[transAmount * k] = np.array(im.crop((3,3,3 + win_size, 3 +win_size)))*1./255.    
        train_x[transAmount * k + 1] = np.array(im.rotate(45).crop((3,3,3 + win_size,3 + win_size)))*1./255.    #print(b0.shape)
        shift0 = random.randint(0,6)
        shift1 = random.randint(0,6)
        shift2 = random.randint(0,6)
        shift3 = random.randint(0,6)
        #shift0 = 3
        #shift1 = 3
        #shift2 = 3
        #shift3 = 3
        if shift0 == 0 and shift1 == 0:
            shift0 += 2
        if shift2 == 0 or shift2 == 3:    
            shift2 +=1
        if shift3 == 0 or shift3 == 3:    
            shift3 +=1    
        train_x[transAmount * k + 2] = np.array(im.crop((shift0,0,shift0 + win_size,win_size)))*1./255.    #print(b0.shape)
        
        train_x[transAmount * k + 3] = np.array(im.crop((0,shift1,win_size,shift1 + win_size)))*1./255.    #print(b0.shape)
        train_x[transAmount * k + 4] = np.array(im.rotate(90).crop((shift2,shift3,shift2 + win_size,shift3 + win_size)))*1./255.
        train_x[transAmount * k + 5] = np.array(im.rotate(180).crop((3,3,3+win_size,3+win_size)))*1./255.
        #train_x[transAmount * k + 4] = np.array(im.crop((0,8,13,21)))*1./255.   
        train_z[k * transAmount] = int(labels[k])
        train_z[transAmount * k + 1] = int(labels[k])
        train_z[transAmount * k + 3] = int(labels[k])
        train_z[transAmount * k + 2] = int(labels[k])   
        train_z[transAmount * k + 4] = int(labels[k])  
        train_z[transAmount * k + 5] = int(labels[k])  
        k+=1
        if k==dlin: break
    temp=[]
    #for i in range(len(train_z)):
       # temp.append(to_categorical(train_z[i],num_classes=len(colors)))
    train_y=np.array(train_z)    
    return train_x,train_y
train_x1,train_y1=load_CNN_train_augment(dir_name)
print(train_y1)
'''
train_x1,train_y1=load_CNN_train_augment(dir_name)
%matplotlib inline
def show_img_data(X_data,K0,rows,cols):   # смотрим на первые rows*cols 
    fig, axes = plt.subplots(rows, cols, figsize=(4, 2), dpi=160) #, figsize=(20, 6)) 
    for j in range(rows):
        for k in range(cols):
            axes[j,k].set_axis_off()
            axes[j,k].imshow(X_data[K0+j*cols+k].squeeze(), cmap='Greys', interpolation='None') #)'auto')

show_img_data(train_x1,0,4,8) 
'''
#np.random.shuffle(train_x1)
#print('train_x_augm.shape=',train_x1.shape)

In [ ]:
amount_valid = 500
k = 0
idx_del = set()
train_x_valid = np.zeros((amount_valid, win_size, win_size, 3), dtype='float32')
train_y_valid = np.zeros((amount_valid,1,1), dtype='float32')
while k < amount_valid:
    idx = random.randint(0, train_x1.shape[0]-1)
    while idx in idx_del:
        idx = random.randint(0, train_x1.shape[0]-1)
    idx_del.add(idx)
    train_x_valid[k] = train_x1[idx]
    train_y_valid[k] = train_y1[idx]
    k += 1



idx_del = list(idx_del)
train_x_data = np.delete(train_x1, idx_del,0)
train_y_data = np.delete(train_y1, idx_del,0)

print('train_x_augm.shape=',train_y_valid.shape)
print('train_x_augm.shape=',train_x_data.shape)
#реорганизация набора тестовых данных, чтобы была равномерность 
train_x_valid_buf = np.zeros((train_x_data.shape[0]//2, win_size, win_size, 3), dtype='float32')
train_y_valid_buf = np.zeros((train_x_data.shape[0]//2,1,1), dtype='float32')

for i in range(train_x_data.shape[0]//2):
    if i%2 ==0:
        train_x_valid_buf[i] = train_x_data[2*i]
        train_y_valid_buf[i] = train_y_data[2*i]
np.flip( train_x_valid_buf)  
np.flip( train_y_valid_buf)  
for i in range(train_x_data.shape[0]//2):
    if i%2 ==0:
        train_x_data[2*i] = train_x_valid_buf[i]
        train_y_data[2*i] = train_y_valid_buf[i]

In [ ]:
num_train, height, width, depth = train_x_data.shape # there are 50000 training examples in CIFAR-10 
print(num_train, height, width, depth)

In [ ]:
accuracy = np.zeros(len(colors), dtype='int')
all_elems = np.zeros(len(colors), dtype='int')
def estimateAccuracy(train_a,train_b,dlin_t,model_t):
     for k in range(dlin_t):
        ttt=np.zeros((1,win_size, win_size,3), dtype ='float32')
        ttt[0]=train_a[k]
        z = model_t.predict(ttt)   
        tx = train_b[k,0]
    
        ty = z; #z[0]
        m_idx = np.argmax(ty[0])  
        real_idx = np.argmax(tx) 
        if  m_idx == real_idx:
            accuracy[real_idx]  += 1
            all_elems[real_idx] += 1
        else:
            all_elems[real_idx] += 1


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import numpy as np


In [ ]:
#(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Предобработаем данные (это массивы Numpy)
#x_train = x_train.reshape(60000, 784).astype('float32') / 255
#x_test = x_test.reshape(10000, 784).astype('float32') / 255

#y_train = y_train.astype('float32')
#y_test = y_test.astype('float32')

# Зарезервируем 10,000 примеров для валидации
#x_val = x_train[-10000:]
#y_val = y_train[-10000:]
# = x_train[:-10000]
#y_train = y_train[:-10000]
kernel_size = 3 # we will use 3x3 kernels throughout
    #pool_size = 2 # we will use 2x2 pooling throughout
conv_depth_1 = 3#height//2 # we will initially have 32 kernels per conv. layer...
conv_depth_2 = height #64 # ...switching to 64 after the first pooling layer
drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
hidden_size = 68 # the FC layer will have 512 neurons
num_classes=5
inputs = keras.Input(shape=(11,11,), name='digits')
#x = layers.Conv2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu', input_shape = (height, width,depth)),(inputs)
#x = layers.Conv2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='tanh')(x[1])
#x = layers.MaxPooling2D(pool_size = (2, 2))(x[1])
#x = layers.Flatten()(x)
#outputs = layers.Dense(num_classes, activation='softmax', name='predictions')(x)
from tensorflow.keras.models import Sequential
#model = keras.Model(inputs=inputs, outputs=outputs)
model = Sequential()
model.add(layers.Conv2D(conv_depth_1+3, kernel_size=3, padding='same', activation='relu', input_shape = (height, width,depth)))
model.add(layers.Conv2D(5+1, kernel_size=3, padding='same', activation='relu'))
model.add(layers.Conv2D(5+1, kernel_size=3, padding='same', activation='relu'))
model.add(layers.Dropout(0.25))
model.add(layers.MaxPooling2D(pool_size = (2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(num_classes,  name='predictions'))
#model.add(Dense(no_classes, activation='softmax'))
# Создадим экземпляр оптимизатора.
optimizer =  keras.optimizers.Adam(learning_rate=0.001)
# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()
model.compile(loss='sparse_categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
# Reserve 10,000 samples for validation.
#x_val = x_train[-10000:]
#y_val = y_train[-10000:]
#x_train = x_train[:-10000]
#y_train = y_train[:-10000]

# Подготовим тренировочный датасет.
batch_size = 24
#train_dataset = tf.data.Dataset.from_tensor_slices((train_x_data, train_y_data))
#train_dataset = train_dataset.shuffle(buffer_size=12002).batch(batch_size)
epochs = 45
# Prepare the validation dataset.
val_dataset = tf.data.Dataset.from_tensor_slices((train_x_valid, train_y_valid))
val_dataset = val_dataset.batch(batch_size)
data1 = np.zeros(epochs, dtype=float) 
data2 = np.zeros(epochs, dtype=float) 
data3 = np.zeros(epochs, dtype=float) 
data4 = np.zeros(epochs, dtype=float) 
data5 = np.zeros(epochs, dtype=float) 
train_dataset = tf.data.Dataset.from_tensor_slices((train_x_data, train_y_data))
train_dataset = train_dataset.shuffle(buffer_size=10000).batch(batch_size)
# Итерируем по эпохам.
val_acc_min=0.0

gradhistory = []
    
def recordweight():
    data = {}
    for g,w in zip(grads, model.trainable_weights):
        if '/kernel:' not in w.name:
            continue # skip bias
        name = w.name.split("/")[0]
        data[name] = g.numpy()
    gradhistory.append(data)
    
for epoch in range(epochs):
    print('Начинаем эпоху %d' % (epoch,))
   
  # Итерируем по пакетам в датасете.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):

    # Откроем GradientTape чтобы записать операции
    # выполняемые во время прямого прохода, включающего автодифференцирование.
        with tf.GradientTape() as tape:

      # Запустим прямой проход слоя.
      # Операции применяемые слоем к своим
      # входным данным будут записаны
      # на GradientTape.
            logits = model(x_batch_train, training=True)  # Logits for this minibatch
            #print(logits)
           # print(y_batch_train)
      # Вычислим значение потерь для этого минибатча.
            loss_value = loss_fn(y_batch_train, logits)

    # Используем gradient tape для автоматического извлечения градиентов
    # обучаемых переменных относительно потерь.
        grads = tape.gradient(loss_value, model.trainable_weights)
        if step == 120:
            print(len(model.trainable_weights))
           # print(model.layers[0].trainable_weights)
            #print(model.trainable_weights[0])
            print(loss_value)
            print(np.mean(abs(np.array(model.trainable_weights[0]))))
            print(np.mean(abs(model.trainable_weights[1])))
            print(np.mean(abs(model.trainable_weights[2])))
            print(np.mean(abs(model.trainable_weights[3])))
            print(np.mean(abs(model.trainable_weights[4])))
            print("=============1==============")
            print(np.mean(abs(grads[1].numpy())), np.mean(abs(grads[1].numpy())/np.mean(abs(model.trainable_weights[0].numpy()))))
            print("=============2==============")
            print(np.mean(abs(grads[2].numpy())), np.mean(abs(grads[2].numpy()))/np.mean(abs(model.trainable_weights[1].numpy())))
            print("=============3==============")
            print(np.mean(abs(grads[3].numpy())), np.mean(abs(grads[3].numpy()))/np.mean(abs(model.trainable_weights[2].numpy())))
            print("=============4==============")
            print(np.mean(abs(grads[4].numpy())), np.mean(abs(grads[4].numpy()))/np.mean(abs(model.trainable_weights[3].numpy())))
            print("=============5==============")
            print(np.mean(abs(grads[5].numpy())), np.mean(abs(grads[5].numpy()))/np.mean(abs(model.trainable_weights[4].numpy())))
            print("=============6==============")
            data1[epoch] = np.mean(abs(grads[1].numpy()))
            data2[epoch] = np.mean(abs(grads[2].numpy()))
            data3[epoch] = np.mean(abs(grads[3].numpy()))
            data4[epoch] = np.mean(abs(grads[4].numpy()))
            data5[epoch] = np.mean(abs(grads[5].numpy()))
        #print(numpy.mean(abs(grads[6].numpy())))
    # Выполним один шаг градиентного спуска обновив
    # значение переменных минимизирующих потери.
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        train_acc_metric.update_state(y_batch_train, logits)
        if step == 0:
            recordweight()
    # Пишем лог каждые 200 пакетов.
        if step % 200 == 0:
        #print('Потери на обучении (для одного пакета) на шаге %s: %s' % (step, float(loss_value)))
            print('Уже увидено: %s примеров' % ((step + 1) * 64))
        #print(grads[].numpy())
    recordweight()
    train_acc = train_acc_metric.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))    
    for x_batch_val, y_batch_val in val_dataset:
        val_logits = model(x_batch_val, training=False)
        # Update val metrics
        val_acc_metric.update_state(y_batch_val, val_logits)
    val_acc = float(val_acc_metric.result())
    if val_acc > val_acc_min:
        model_temp = model
        val_acc_min=val_acc
    val_acc_metric.reset_states()
    print("Validation acc: %.4f" % (float(val_acc),))
    #print("Time taken: %.2fs" % (time.time() - start_time))

In [ ]:
fig, ax = plt.subplots(3, 1, sharex=True, constrained_layout=True, figsize=(8, 12))
ax[0].set_title("Mean gradient")
for key in gradhistory[0]:
    ax[0].plot(range(len(gradhistory)), [w[key].mean() for w in gradhistory], label=key)
ax[0].legend()
plt.show()

In [ ]:
'''
print("grad_shape = ", len(grads))
import matplotlib.pyplot as plt
gridsize = (1, 2)
fig = plt.figure(figsize=(18, 4))
ax1 = plt.subplot2grid(gridsize, (0, 0))
ax2 = plt.subplot2grid(gridsize, (0, 1))
#data = np.loadtxt("C:/Users/anastasya/Desktop/data_deviation.txt").flatten()
#print(data)
#data = np.array([78.4,81.41,80.25,76.0,78.45], dtype = float)
#ax1.set_title(title1 )
#ax1.hist(data.flatten(), density=True)


x = np.linspace(1, epochs, epochs)
#ax1.stem(x,data-40)#ax1.spines["top"].set_alpha(.3)
ax1.grid()
ax1.set_xlabel("Номер эпохи")
ax1.set_ylabel("Величина отклонения градиента")

ax1.plot(x, data1, color='red')
ax1.plot(x, data3, color='blue')
ax1.plot(x, data5, color='green')
plt.show()
'''

In [ ]:
temple_file = Image.open(img_file_name)
img = Image.new('RGB', temple_file.size, color=0)
x0 = 0
y0 = 0
draw  = ImageDraw.Draw(img)
width, height = img.size
im2=np.zeros((1, win_size, win_size, 3), dtype='float32')
class_marks = np.zeros((width,height), dtype='int')
model = model_temp
for i in range(0, width,step_len -1):
    for j in range(0, height,step_len -1):
        if(i + win_size) < width and (j + win_size ) < height:
            x2y = [(i,j), (i + win_size, j),(i + win_size, j + win_size), (i, j + win_size)]
            im2[0] = np.array(  temple_file.crop((i, j, i + win_size,j + win_size)))*1./255.
            z = model.predict(im2)     
            #print('z   ', z)
            m_idx = np.argmax(z[0])
            for k in range(win_size):
                for l in range(win_size):
                    class_marks[i + k, j + l] = m_idx #+ 1
            draw.polygon(x2y, outline = t_color[m_idx], fill = t_color[m_idx])

img.save(result_fileName)
img.close()

In [ ]:
with open(marks_filename, 'w') as file:
    for i in range(img_len):
        for j in range(img_len):
            file.write(str(class_marks[j, img_len-i] +1)) 
            file.write(" ") 
        file.write("\n")
    file.close()   

In [ ]:
gridsize = (1, 2)
fig = plt.figure(figsize=(18, 16))
ax1 = plt.subplot2grid(gridsize, (0, 0))
ax2 = plt.subplot2grid(gridsize, (0, 1))
data = np.loadtxt(marks_filename)
pc = ax1.contourf(data)
plt.colorbar(pc, ax=ax1, format='$%d')
plt.show()

In [ ]:
import subprocess
subprocess.run(['C:/Users/anastasya/CMakeBuilds/71f94088-de5b-a431-82e1-cdd7ec577f63/build/x64-Debug (по умолчанию)/exec/exec.exe',"1", metrics, marks_filename, results, faults])